# **데이터 누수**
* ### **Target leakage**
    예측 시점에서 사용할 수 없는 데이터가 데이터 셋에 포함되어 있을 때 발생  
    * ex) 
    * 폐렴에 걸린 사람은 회복을 위해 항생제를 투여한다는 상관관계를 발견 
    * 이 사람이 폐렴에 걸릴지 예측하기위해 항생제를 투여한 사람을 찾음 
    * train, validation에서는 좋은 점수를 얻음
    * 실제에서는 폐렴을 걸려야 항생제를 투약하기 때문에 폐렴에 걸릴지 예측하는과정에서는 매우 부적절함
    * 따라서 타겟 값이 결정된 후 만들어진 데이터들은 모두 제외 (즉, 폐렴이 걸린 후 일어난 모든 일은 모델을 만들때 제외시켜야함 == 항생제 데이터)  

* ### **Train-Test Contamination**
    Train data와 Test data가 제대로 구별되지 않아서 발생
    * Test data가 Train 과정에 개입함
    * 이 과정에서 매우 성능이 높은 모델로 평가됨
    * 하지만 실제 배포 시 좋지 않은 모델로 평가될 수도 있음
    * 이런 문제를 파이프라인이나 다른 것들을 통해 확실히 하는 것이 중요함

In [12]:
import pandas as pd

data = pd.read_csv("../datafile/AER_credit_card_data.csv", true_values = ["yes"], false_values = ["no"])

y = data.card

X = data.drop(["card"], axis=1) 

print(X.shape[0])
X.head()



1319


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


* true_values, false_values : true로 간주할 값, false로 간주할 값

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

my_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))
cv_scores = cross_val_score(my_pipeline, X, y, cv=5, scoring="accuracy")
print(cv_scores.mean())

0.981049083995852


In [13]:
potential_leaks = ["expenditure", "share", "active", "majorcards"]
X2 = X.drop(potential_leaks, axis=1)

cv_scores = cross_val_score(my_pipeline, X2, y, cv=5, scoring="accuracy")

print(cv_scores.mean())

0.8278920382532551
